<a href="https://colab.research.google.com/github/Gustavo0501/Trabalho_Grafos/blob/main/Trabalho_Grafos_Etapa_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import math
import sys
import copy
import time
import traceback

class Rota:
    """Representa a rota de um único veículo, com seus custos e serviços."""
    def __init__(self, id_rota, deposito):
        self.id_rota = id_rota
        self.deposito = deposito
        self.carga_total = 0
        self.custo_total = 0
        self.servicos_atendidos = []
        self.posicao_atual = deposito
        self.sequencia_visitas = [f"(D 0,{deposito},{deposito})"]

    def adicionar_servico(self, servico, custo_deslocamento, ponto_entrada, ponto_saida):
        """Adiciona um serviço à rota e atualiza os totais de custo e carga."""
        self.carga_total += servico['demanda']
        self.custo_total += custo_deslocamento + servico['custo']
        self.posicao_atual = ponto_saida
        self.servicos_atendidos.append(servico)
        self.sequencia_visitas.append(f"(S {servico['id']},{ponto_entrada},{ponto_saida})")

    def finalizar_rota(self, distancias):
        """Calcula o custo de retorno ao depósito e completa a sequência de visitas."""
        if self.posicao_atual is not None:
            self.custo_total += distancias[self.posicao_atual][self.deposito]
        self.sequencia_visitas.append(f"(D 0,{self.deposito},{self.deposito})")

    def to_string(self):
        """Formata a rota como uma string para o arquivo de solução."""
        return f" 0 1 {self.id_rota} {self.carga_total} {int(self.custo_total)} {len(self.sequencia_visitas)} {' '.join(self.sequencia_visitas)}"

class Solucao:
    """Armazena o conjunto de todas as rotas que compõem a solução final."""
    def __init__(self):
        self.rotas = []
        self.custo_total = 0

    def recalcular_custo_total(self):
        """Recalcula o custo total da solução a partir de suas rotas."""
        self.custo_total = sum(r.custo_total for r in self.rotas)

    def salvar_em_arquivo(self, nome_arquivo_instancia, pasta_saida, tempo_total_ns, tempo_encontrar_ns):
        """Salva a solução completa em um arquivo, seguindo o formato padrão."""
        os.makedirs(pasta_saida, exist_ok=True)
        caminho_saida = os.path.join(pasta_saida, f"sol-{os.path.splitext(nome_arquivo_instancia)[0]}.dat")
        with open(caminho_saida, 'w') as f:
            custo_final = sum(r.custo_total for r in self.rotas)
            f.write(f"{int(custo_final)}\n")
            f.write(f"{len(self.rotas)}\n")
            f.write(f"{tempo_total_ns}\n")
            f.write(f"{tempo_encontrar_ns}\n")
            for rota in self.rotas:
                f.write(f"{rota.to_string()}\n")
        print(f"Solução salva em: {caminho_saida}")

class Grafo:
    """Lê, armazena e pré-processa todos os dados de uma instância."""
    def __init__(self, caminho_arquivo):
        self.nome_arquivo = os.path.basename(caminho_arquivo)
        self.vertices, self.deposito, self.capacidade, self.max_veiculos = 0, None, 0, -1
        self.arestas, self.arcos, self.servicos_requeridos = [], [], []
        self.distancias, self.predecessores = [], []
        self._ler_arquivo_instancia(caminho_arquivo)
        self._calcular_caminhos_curtos()

    def _ler_arquivo_instancia(self, caminho_arquivo):
        """Lê o arquivo de instância de forma robusta, preenchendo os atributos do grafo."""
        with open(caminho_arquivo, 'r') as f:
            linhas = [linha.strip() for linha in f if linha.strip() and not linha.lower().startswith("based")]
        secao, id_servico_counter = None, 1
        for linha in linhas:
            if linha == "-1": continue

            # Leitura do cabeçalho
            if ':' in linha:
                chave, valor = [p.strip() for p in linha.split(':')]
                if chave == 'Depot Node': self.deposito = int(valor)
                elif chave == '#Nodes': self.vertices = int(valor)
                elif chave == 'Capacity': self.capacidade = int(valor)
                elif chave == '#Vehicles': self.max_veiculos = int(valor)
            # Identificação da seção de dados
            elif linha.startswith('ReN.'): secao = 'ReN'
            elif linha.startswith('ReE.'): secao = 'ReE'
            elif linha.startswith('ReA.'): secao = 'ReA'
            elif linha.startswith('EDGE'): secao = 'EDGE'
            elif linha.startswith('ARC'): secao = 'ARC'
            else:
                partes = linha.split()
                if not partes: continue
                # Tratamento robusto para leitura dos dados de cada seção
                try:
                    if secao == 'ReN':
                        no, demanda, custo = int(partes[0][1:]), int(partes[1]), int(partes[2])
                        self.servicos_requeridos.append({'id': id_servico_counter,'tipo': 'V','nos': (no, no),'custo': custo,'demanda': demanda})
                        id_servico_counter += 1
                    elif secao == 'ReE':
                        u, v, custo, demanda = int(partes[1]), int(partes[2]), int(partes[3]), int(partes[4])
                        self.servicos_requeridos.append({'id': id_servico_counter,'tipo': 'E','nos': (u, v),'custo': custo,'demanda': demanda})
                        self.arestas.append((u, v, custo)); id_servico_counter += 1
                    elif secao == 'ReA':
                        u, v, custo, demanda = int(partes[1]), int(partes[2]), int(partes[3]), int(partes[4])
                        self.servicos_requeridos.append({'id': id_servico_counter,'tipo': 'A','nos': (u, v),'custo': custo,'demanda': demanda})
                        self.arcos.append((u, v, custo)); id_servico_counter += 1
                    elif secao == 'EDGE': self.arestas.append((int(partes[1]), int(partes[2]), int(partes[3])))
                    elif secao == 'ARC': self.arcos.append((int(partes[1]), int(partes[2]), int(partes[3])))
                except (ValueError, IndexError):
                    # Ignora linhas malformadas ou cabeçalhos de seções
                    continue

    def _calcular_caminhos_curtos(self):
        """Executa o algoritmo de Floyd-Warshall para preencher a matriz de distâncias."""
        V = self.vertices
        self.distancias = [[math.inf] * (V + 1) for _ in range(V + 1)]
        for i in range(1, V + 1): self.distancias[i][i] = 0
        for u, v, custo in self.arestas:
            if custo < self.distancias[u][v]:
                self.distancias[u][v] = self.distancias[v][u] = custo
        for u, v, custo in self.arcos:
            if custo < self.distancias[u][v]:
                self.distancias[u][v] = custo
        # Algoritmo principal de Floyd-Warshall
        for k in range(1, V + 1):
            for i in range(1, V + 1):
                for j in range(1, V + 1):
                    if self.distancias[i][k] + self.distancias[k][j] < self.distancias[i][j]:
                        self.distancias[i][j] = self.distancias[i][k] + self.distancias[k][j]
        print(f"Matriz de distâncias calculada para {self.nome_arquivo}.")

class PathScanning:
    """Heurística construtiva para frotas ILIMITADAS (veiculos = -1)."""
    def __init__(self, grafo):
        self.grafo = grafo

    def executar(self):
        """Constrói a solução sequencialmente, preenchendo uma rota ao máximo antes de criar a próxima."""
        solucao = Solucao()
        servicos_nao_atendidos = set(s['id'] for s in self.grafo.servicos_requeridos)
        servicos_map = {s['id']: s for s in self.grafo.servicos_requeridos}
        rota_id = 1
        while servicos_nao_atendidos:
            rota_atual = Rota(rota_id, self.grafo.deposito)
            # Constrói uma rota adicionando o serviço mais próximo sucessivamente
            while True:
                candidatos = []
                for sid in servicos_nao_atendidos:
                    servico = servicos_map[sid]
                    if rota_atual.carga_total + servico['demanda'] <= self.grafo.capacidade:
                        u, v = servico['nos']
                        dist_u = self.grafo.distancias[rota_atual.posicao_atual][u]
                        dist_v = self.grafo.distancias[rota_atual.posicao_atual][v]
                        # Para arestas (não direcionadas), escolhe o ponto de entrada mais próximo
                        if servico['tipo'] == 'E' and dist_v < dist_u:
                            custo_desloc, p_ent, p_sai = dist_v, v, u
                        else:
                            custo_desloc, p_ent, p_sai = dist_u, u, v
                        candidatos.append((custo_desloc, servico, p_ent, p_sai))
                if not candidatos: break

                # Escolha gulosa: seleciona o candidato com menor custo de deslocamento
                candidatos.sort(key=lambda x: x[0])
                melhor_custo, melhor_servico, entrada, saida = candidatos[0]

                rota_atual.adicionar_servico(melhor_servico, melhor_custo, entrada, saida)
                servicos_nao_atendidos.remove(melhor_servico['id'])

            rota_atual.finalizar_rota(self.grafo.distancias)
            solucao.rotas.append(rota_atual)
            rota_id += 1
        solucao.recalcular_custo_total()
        return solucao

class InsertionHeuristic:
    """Heurística construtiva para frotas FIXAS (veiculos > 0)."""
    def __init__(self, grafo):
        self.grafo = grafo

    def executar(self):
        """Constrói a solução distribuindo os serviços entre um número alvo de rotas."""
        solucao = Solucao()
        servicos_nao_atendidos = list(self.grafo.servicos_requeridos)

        # Inicia com k rotas se a frota for fixa, ou 0 se for ilimitada
        num_rotas_iniciais = self.grafo.max_veiculos if self.grafo.max_veiculos > 0 else 0
        for i in range(num_rotas_iniciais):
            solucao.rotas.append(Rota(i + 1, self.grafo.deposito))

        while servicos_nao_atendidos:
            melhor_aumento_custo, melhor_servico, melhor_rota, melhor_pos = float('inf'), None, None, -1

            # Se não houver rotas (caso inicial de frota ilimitada), cria a primeira
            if not solucao.rotas:
                solucao.rotas.append(Rota(1, self.grafo.deposito))

            # Procura a melhor inserção possível em todas as rotas e posições
            for servico in servicos_nao_atendidos:
                for rota in solucao.rotas:
                    if rota.carga_total + servico['demanda'] <= self.grafo.capacidade:
                        for j in range(len(rota.servicos_atendidos) + 1):
                            aumento_custo = self._calcular_custo_insercao(rota, servico, j)
                            if aumento_custo < melhor_aumento_custo:
                                melhor_aumento_custo, melhor_servico, melhor_rota, melhor_pos = aumento_custo, servico, rota, j

            # Se não encontrou lugar nas rotas existentes, cria uma nova para garantir a solução completa
            if melhor_rota is None:
                nova_rota = Rota(len(solucao.rotas) + 1, self.grafo.deposito)
                solucao.rotas.append(nova_rota)
                continue

            # Realiza a melhor inserção encontrada
            melhor_rota.servicos_atendidos.insert(melhor_pos, melhor_servico)
            melhor_rota.carga_total += melhor_servico['demanda']
            servicos_nao_atendidos.remove(melhor_servico)

        # Recalcula custos e sequências finais para garantir consistência
        for rota in solucao.rotas:
            custo, seq = self._recalcular_custo_completo_rota(rota.servicos_atendidos)
            rota.custo_total, rota.sequencia_visitas = custo, seq
        solucao.recalcular_custo_total()

        return solucao

    def _calcular_custo_insercao(self, rota, servico, pos):
        """Calcula o aumento de custo ao inserir um serviço em uma posição de uma rota."""
        u, v, custo_servico = servico['nos'][0], servico['nos'][1], servico['custo']
        no_ant = rota.servicos_atendidos[pos-1]['nos'][1] if pos > 0 else self.grafo.deposito
        no_prox = rota.servicos_atendidos[pos]['nos'][0] if pos < len(rota.servicos_atendidos) else self.grafo.deposito
        custo_antigo = self.grafo.distancias[no_ant][no_prox]
        custo_novo = self.grafo.distancias[no_ant][u] + custo_servico + self.grafo.distancias[v][no_prox]
        return custo_novo - custo_antigo

    def _recalcular_custo_completo_rota(self, lista_servicos):
        """Recalcula o custo e a sequência de uma rota do zero."""
        custo, pos_atual = 0, self.grafo.deposito
        seq_visitas = [f"(D 0,{self.grafo.deposito},{self.grafo.deposito})"]
        for servico in lista_servicos:
            p_ent, p_sai = servico['nos']
            custo += self.grafo.distancias[pos_atual][p_ent] + servico['custo']
            pos_atual = p_sai
            seq_visitas.append(f"(S {servico['id']},{p_ent},{p_sai})")
        custo += self.grafo.distancias[pos_atual][self.grafo.deposito]
        seq_visitas.append(f"(D 0,{self.grafo.deposito},{self.grafo.deposito})")
        return custo, seq_visitas

class BuscaLocal:
    """(Etapa 3) Refina uma solução com movimentos de vizinhança."""
    def __init__(self, grafo):
        self.grafo = grafo
        self.solucao_atual = None

    def executar(self, solucao_inicial):
        """Executa o processo de busca local até um ótimo local ser atingido."""
        self.solucao_atual = copy.deepcopy(solucao_inicial)
        print(f"\n--- Iniciando Busca Local... Custo inicial: {int(self.solucao_atual.custo_total)} ---")

        iteracao = 0
        while True:
            iteracao += 1
            custo_antes_da_iteracao = self.solucao_atual.custo_total

            # A estratégia é 'first improvement': o primeiro movimento de melhora é aceito.
            if self._tentar_transferencia():
                print(f"  Iteração {iteracao} | Custo melhorado (transferência): {int(self.solucao_atual.custo_total)}")
                continue

            if self._tentar_re_insercao_intra_rota():
                print(f"  Iteração {iteracao} | Custo melhorado (re-inserção): {int(self.solucao_atual.custo_total)}")
                continue

            # Se nenhum movimento melhorou a solução, o ótimo local foi atingido.
            print(f"  Iteração {iteracao} | Nenhuma melhoria encontrada.")
            break

        print(f"Busca Local finalizada. Custo final: {int(self.solucao_atual.custo_total)}")
        return self.solucao_atual

    def _recalcular_custo_e_sequencia_rota(self, lista_servicos):
        """Recalcula o custo e a sequência de uma rota do zero."""
        custo, pos_atual = 0, self.grafo.deposito
        seq_visitas = [f"(D 0,{self.grafo.deposito},{self.grafo.deposito})"]
        for servico in lista_servicos:
            p_ent, p_sai = servico['nos']
            custo += self.grafo.distancias[pos_atual][p_ent] + servico['custo']
            pos_atual = p_sai
            seq_visitas.append(f"(S {servico['id']},{p_ent},{p_sai})")
        custo += self.grafo.distancias[pos_atual][self.grafo.deposito]
        seq_visitas.append(f"(D 0,{self.grafo.deposito},{self.grafo.deposito})")
        return custo, seq_visitas

    def _tentar_transferencia(self):
        """Tenta mover um serviço de uma rota para outra."""
        for r_origem in self.solucao_atual.rotas:
            for i in range(len(r_origem.servicos_atendidos) - 1, -1, -1):
                servico = r_origem.servicos_atendidos[i]
                for r_destino in self.solucao_atual.rotas:
                    if r_origem.id_rota == r_destino.id_rota: continue
                    if r_destino.carga_total + servico['demanda'] > self.grafo.capacidade: continue

                    custo_antigo = r_origem.custo_total + r_destino.custo_total
                    serv_origem_temp = r_origem.servicos_atendidos[:i] + r_origem.servicos_atendidos[i+1:]
                    novo_custo_origem, _ = self._recalcular_custo_e_sequencia_rota(serv_origem_temp)
                    melhor_custo_destino, melhor_pos = float('inf'), -1

                    for j in range(len(r_destino.servicos_atendidos) + 1):
                        serv_destino_temp = r_destino.servicos_atendidos[:j] + [servico] + r_destino.servicos_atendidos[j:]
                        custo_teste, _ = self._recalcular_custo_e_sequencia_rota(serv_destino_temp)
                        if custo_teste < melhor_custo_destino:
                            melhor_custo_destino, melhor_pos = custo_teste, j

                    if novo_custo_origem + melhor_custo_destino < custo_antigo:
                        r_origem.servicos_atendidos.pop(i)
                        r_origem.carga_total -= servico['demanda']
                        r_origem.custo_total, r_origem.sequencia_visitas = self._recalcular_custo_e_sequencia_rota(r_origem.servicos_atendidos)

                        r_destino.servicos_atendidos.insert(melhor_pos, servico)
                        r_destino.carga_total += servico['demanda']
                        r_destino.custo_total, r_destino.sequencia_visitas = self._recalcular_custo_e_sequencia_rota(r_destino.servicos_atendidos)

                        self.solucao_atual.recalcular_custo_total()
                        return True
        return False

    def _tentar_re_insercao_intra_rota(self):
        """Tenta mover um serviço para uma posição diferente dentro da mesma rota."""
        for rota in self.solucao_atual.rotas:
            if len(rota.servicos_atendidos) < 2: continue
            for i in range(len(rota.servicos_atendidos)):
                servico = rota.servicos_atendidos[i]
                servicos_temp = rota.servicos_atendidos[:i] + rota.servicos_atendidos[i+1:]
                custo_original = rota.custo_total
                for j in range(len(servicos_temp) + 1):
                    if i == j: continue
                    servicos_teste = servicos_temp[:j] + [servico] + servicos_temp[j:]
                    novo_custo, _ = self._recalcular_custo_e_sequencia_rota(servicos_teste)
                    if novo_custo < custo_original:
                        rota.servicos_atendidos = servicos_teste
                        rota.custo_total, rota.sequencia_visitas = self._recalcular_custo_e_sequencia_rota(rota.servicos_atendidos)
                        self.solucao_atual.recalcular_custo_total()
                        return True
        return False

def main():
    """Função principal que orquestra a execução para todas as instâncias."""
    pasta_entrada, pasta_saida = 'MCGRP/', 'G13/'

    if not os.path.isdir(pasta_entrada):
        print(f"Erro: Pasta de entrada '{pasta_entrada}' não encontrada.")
        sys.exit(1)

    for nome_arquivo in [f for f in os.listdir(pasta_entrada) if f.endswith('.dat')]:
        print(f"\n--- Processando instância: {nome_arquivo} ---")
        try:
            inicio_ns = time.perf_counter_ns()
            caminho_completo = os.path.join(pasta_entrada, nome_arquivo)
            grafo = Grafo(caminho_completo)

            # Lógica Híbrida: Escolhe a heurística construtiva mais apropriada para a instância
            if grafo.max_veiculos > 0:
                print("Instância com frota fixa. Usando Heurística de Inserção.")
                algoritmo_construtivo = InsertionHeuristic(grafo)
            else:
                print("Instância com frota ilimitada. Usando Heurística Path Scanning.")
                algoritmo_construtivo = PathScanning(grafo)

            solucao_inicial = algoritmo_construtivo.executar()

            fim_etapa2_ns = time.perf_counter_ns()
            tempo_encontrar_ns = fim_etapa2_ns - inicio_ns

            print(f"Custo da solução inicial: {int(solucao_inicial.custo_total)}")

            # Etapa 3: Refina a solução com a Busca Local
            busca_local = BuscaLocal(grafo)
            solucao_final = busca_local.executar(solucao_inicial)

            fim_total_ns = time.perf_counter_ns()
            tempo_total_ns = fim_total_ns - inicio_ns

            solucao_final.salvar_em_arquivo(nome_arquivo, pasta_saida, tempo_total_ns, tempo_encontrar_ns)

        except Exception as e:
            print(f"Ocorreu um erro ao processar {nome_arquivo}: {e}")
            traceback.print_exc()

if __name__ == "__main__":
    main()


# Readme

In [ ]:
# Projeto de Algoritmos em Grafos - Roteamento de Veículos (MCGRP)

**Universidade Federal de Lavras (UFLA)**
**Disciplina:** GCC262 - Grafos e suas Aplicações
**Professor:** Mayron César O. Moreira

**Integrantes da dupla:** Alexandre Marques Spinola, Gustavo do Carmo Resende

---

## 1. Visão Geral do Projeto

Este projeto aborda uma variação do Problema do Carteiro Chinês Misto (MCGRP), um desafio de otimização combinatória com grande aplicação em logística. O objetivo é desenvolver um sistema para encontrar um conjunto de rotas de custo mínimo para uma frota de veículos, que deve atender a uma série de serviços distribuídos em um grafo (em nós, arestas ou arcos).

A solução foi implementada em Python 3 e segue a metodologia de duas fases principais:
1.  **Fase Construtiva (Etapa 2):** Geração de uma solução inicial válida através de uma heurística inteligente.
2.  **Fase de Melhoria (Etapa 3):** Refinamento da solução inicial utilizando um algoritmo de busca local para otimizar o custo total.

## 2. Estrutura e Arquitetura do Código

O código foi projetado de forma modular e orientado a objetos para garantir clareza, manutenibilidade e um baixo acoplamento entre os componentes lógicos.

* `main.py`: O ponto de entrada do programa. Ele gerencia o fluxo de execução, lendo cada arquivo de instância, acionando os algoritmos e salvando a solução final.

### Classes Principais:

* **`Grafo`**: A classe responsável por toda a interação com os dados. Ela realiza a leitura dos arquivos de instância `.dat`, armazena as informações do problema (capacidade, depósito, serviços, etc.) e executa o pré-processamento essencial, que é o cálculo da matriz de distâncias mínimas entre todos os nós via algoritmo de **Floyd-Warshall**.

* **`Rota`**: Modela a rota de um único veículo. É responsável por controlar a sequência de visitas, o custo acumulado e a carga total, assegurando que as restrições sejam cumpridas.

* **`Solucao`**: Representa a solução completa para uma instância, contendo um conjunto de objetos `Rota`. Ela calcula o custo total e formata o arquivo de saída `.dat` conforme o padrão exigido.

* **`PathScanning`** e **`InsertionHeuristic`** (Etapa 2): São as duas heurísticas construtivas utilizadas para gerar a solução inicial. O sistema adota uma **estratégia híbrida**, escolhendo a heurística mais apropriada com base nas características da instância.

* **`BuscaLocal`** (Etapa 3): Implementa o algoritmo de melhoria. Após a geração de uma solução inicial, esta classe tenta otimizá-la iterativamente, aplicando movimentos de vizinhança para reduzir o custo total.

## 3. Algoritmos Implementados

A solução final é o resultado da combinação de uma heurística construtiva com uma busca local.

### Etapa 2: Estratégia Híbrida para Construção da Solução

O programa analisa cada instância e escolhe a melhor ferramenta para o trabalho:

1.  **Para Frotas Ilimitadas (`#Vehicles: -1`):**
    * **Algoritmo:** `PathScanning`.
    * **Lógica:** Esta heurística é puramente gulosa e sequencial. Ela preenche uma rota ao máximo, sempre escolhendo o serviço mais próximo, antes de criar uma nova rota.
    * **Justificativa:** Essa abordagem naturalmente minimiza o número de veículos utilizados, o que é ideal quando não há um limite de frota, pois cada rota a menos economiza o custo de uma viagem de ida e volta ao depósito.

2.  **Para Frotas Fixas (`#Vehicles > 0`):**
    * **Algoritmo:** `InsertionHeuristic`.
    * **Lógica:** Esta heurística é mais estratégica. Ela considera todas as rotas disponíveis e, para cada serviço restante, encontra a melhor posição de inserção (aquela que causa o menor aumento de custo) em qualquer uma das rotas.
    * **Justificativa:** É superior para distribuir os serviços de forma mais equilibrada entre um número predefinido de veículos. Caso seja impossível atender a todos os serviços sem exceder o limite da frota, o algoritmo tem a permissão de criar rotas extras para garantir que a solução seja sempre completa.

### Etapa 3: Busca Local para Melhoria

Após a construção da solução inicial, um algoritmo de **Busca Local** é aplicado para refiná-la. Ele opera com a estratégia de *first improvement* (primeira melhora) e explora duas vizinhanças clássicas, em sequência:

1.  **Transferência (Relocate Inter-Rotas):** Tenta mover um serviço de sua rota atual para uma posição em outra rota.
2.  **Re-inserção (Relocate Intra-Rota):** Tenta encontrar uma posição melhor para um serviço dentro de sua própria rota.

O processo continua até que nenhum dos dois movimentos consiga mais reduzir o custo da solução, atingindo assim um ótimo local.

## 4. Como Executar o Projeto

### Pré-requisitos
* Python 3.x

### Passos
1.  Assegure que todas as dependências (`os`, `math`, `sys`, `copy`, `time`, `traceback`, `random`) estejam disponíveis na sua instalação Python (são todas bibliotecas padrão).
2.  Crie uma pasta chamada `MCGRP` no mesmo diretório do script e adicione todos os arquivos de instância (`.dat`) nela.
3.  Defina o nome da pasta de saída na função `main()`. O padrão atual é `G13`.
    ```python
    # Dentro da função main()
    pasta_entrada, pasta_saida = 'MCGRP/', 'G13/'
    ```
4.  Execute o script principal pelo terminal:
    ```bash
    python nome_do_arquivo.py
    ```
O script irá iterar sobre cada instância, aplicar os algoritmos e salvar a solução final otimizada na pasta de saída designada.

## 5. Formato do Arquivo de Saída

Cada arquivo `sol-*.dat` gerado segue estritamente o formato:
* **Linha 1:** Custo total da solução final.
* **Linha 2:** Número total de rotas na solução.
* **Linha 3:** Tempo total de execução do algoritmo em nanossegundos (Etapa 2 + Etapa 3).
* **Linha 4:** Tempo para gerar a solução inicial em nanossegundos (apenas Etapa 2).
* **Linhas seguintes:** A descrição detalhada de cada rota.
